In [20]:
import tensorflow as tf
model_path = "hourse_car_person.h5"
tf_model =  tf.keras.models.load_model(model_path)
tf_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                    

In [3]:
import os
train_path = os.getcwd()+"\\dataset\\train"
validation_path = os.getcwd()+"\\dataset\\val"

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_path,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(256, 256))     

validation_generator =  test_datagen.flow_from_directory(validation_path,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         target_size=(256, 256))

Found 540 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


# Tensorflow model Acc

In [ ]:

import numpy as np
class_label = ['car','hourse','person']
true_labels = []
predicted_labels = []

import matplotlib.pyplot as plt

count = 0
for image,label in validation_generator:
    
    
    
    predicted_temp = tf_model.predict(image)
    for i in range(len(label)):
        true_labels.append(class_label[np.argmax(label[i])])
        predicted_labels.append(class_label[np.argmax(predicted_temp[i])])
        

    count = count+1
    if count > 2:
        break
    
test_accuracy = 100 * np.sum(np.array(predicted_labels)==true_labels) / len(predicted_labels)
print("TF model Test accuracy : {}".format(test_accuracy))

## TF model -> tflite model

In [10]:

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()

with open('hourse_car_person.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\tharh\AppData\Local\Temp\tmp2lxp6ygb\assets


INFO:tensorflow:Assets written to: C:\Users\tharh\AppData\Local\Temp\tmp2lxp6ygb\assets


In [34]:
# Load TFLite model and see some details about input/output

tflite_interpreter = tf.lite.Interpreter(model_path="hourse_car_person.tflite")
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: serving_default_conv2d_input:0
shape: [  1 256 256   3]
type: <class 'numpy.float32'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [1 3]
type: <class 'numpy.float32'>


In [42]:
import numpy as np
class_label = ['car','hourse','person']
true_labels = []
predicted_labels = []


count = 0
for image,label in validation_generator:
    
    
    

    for i in range(len(label)):
        current_image = image[i]
        #print(current_image.shape)
        current_image = np.array([current_image])
        #print(current_image.shape)
        
        # Run inference
        tflite_interpreter.set_tensor(input_details[0]['index'], current_image)
        tflite_interpreter.invoke()
        tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
        #print("Prediction results shape:", tflite_model_predictions.shape)
       
        true_labels.append(class_label[np.argmax(label[i])])
        predicted_labels.append(class_label[np.argmax(tflite_model_predictions[0])])
        

    count = count+1
    if count > 2:
        break
        
test_accuracy = 100 * np.sum(np.array(predicted_labels)==true_labels) / len(predicted_labels)
print("TF lite model Test accuracy : {}".format(test_accuracy))

TF lite model Test accuracy : 75.0
